In [1]:
import pandas as pd
import numpy as np
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    IntegerType, StringType, IntegerType, FloatType, 
    StructField, StructType, DoubleType
)
from pyspark.ml.recommendation import ALS
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pyspark.sql.functions import udf, col, isnan
import psycopg2
from pandas.io import sql
from sklearn.model_selection import train_test_split
from pyspark.ml.evaluation import RegressionEvaluator
from itertools import product

In [2]:
cd ../mixmaker/

/Users/brettashley/galvanize/dsi/00-Capstone/ml-mix-lab/mixmaker


In [3]:
import database_interaction
import model

In [ ]:
df = db.get_table('connections_distinct')
data = sr.spark.createDataFrame(df)

In [ ]:
train, test = data.randomSplit([0.95, 0.05])

In [ ]:
sr = model.SongRecommender(itemCol='song_id',
                     userCol='sampled_by_song_id',
                     ratingCol='is_connected',
                     nonnegative=True,
                     regParam=0.01,
                     rank=200,
                     alpha=1)
db = database_interaction.DatabaseInteraction(db_name='mixmaker')

In [ ]:
recommender = sr.fit(train)

In [ ]:
preds = sr.transform(recommender, test)

In [ ]:
sr.rmse(preds, True)

In [ ]:
preds_train = sr.transform(recommender, train)

In [ ]:
sr.rmse(preds_train)

In [ ]:
preds.filter("prediction + 1 > prediction").count()

In [ ]:
preds.count()

In [ ]:
top_10_recs = recommender.recommendForAllUsers(10).toPandas()

In [ ]:
len(top_10_recs)

In [ ]:
train.count()

In [ ]:
top_10_recs.head()

In [ ]:
top_10_recs.loc[0,'recommendations']

In [ ]:
for i, row in top_10_recs.head().iterrows():
    print(list([row[0]] * 10))
    print((pd.DataFrame(row['recommendations'])))

In [ ]:
output = pd.DataFrame(columns=['user_song_id', 'item_song_id', 'rating'])

In [ ]:
output

In [ ]:
for i, row in top_10_recs.head().iterrows():
    df = pd.DataFrame(row[1], columns=['item_song_id', 'rating'])
    df['user_song_id'] = list([row[0]] * 10)
    output = pd.concat([output, df])

In [ ]:
recommender = sr.fit(data)

In [ ]:
preds = sr.get_predictions_for_all_users(recommender)

In [ ]:
db = database_interaction.DatabaseInteraction()

In [ ]:
preds.columns

In [ ]:
','.join((preds.loc[:,'user_song_id'].values))

In [ ]:
query = """
        INSERT INTO predictions_temp (user_song_id, item_song_id, rating)
        SELECT %s, %s, %s
        """

In [ ]:
for i, row in preds.head().iterrows():
    if i % 2 == 0:
        print(i)
    print(i, row[0], row[1], row[2])

In [ ]:
test = preds[:100].reset_index(drop=True)

In [ ]:
preds_reset = preds.reset_index(drop=True)

In [ ]:
len(preds_reset)

In [ ]:
for row in test.itertuples():
    if row[0] % 10 == 0:
        print(row[0])
    print(row[0], row[1], row[2], row[3])

In [ ]:
for row in preds_reset.head().itertuples():
    if row[0] % 5000 == 0:
        print(row[0])
    db.cur.execute(query, 
                   (row[1], row[2], row[3]))
    db.conn.commit()

In [ ]:
for row in preds_reset.itertuples():
    if row[0] % 5000 == 0:
        print(row[0])
    db.cur.execute(query, 
                   (row[1], row[2], row[3]))
    db.conn.commit()

In [ ]:
df = data.toPandas()

In [ ]:
df.columns

In [ ]:
existing_pairs = set((x,y) for 
                     [x,y] in df[['song_id', 'sampled_by_song_id']].values)

In [ ]:
len(existing_pairs)

In [ ]:
for row in test.itertuples():
    if (row[1], row[2]) in existing_pairs:
        test.drop(row[0], inplace=True)

In [ ]:
test

In [ ]:
sr = model.SongRecommender()

In [ ]:
train, test = data.randomSplit([.995, .005])

In [ ]:
recommender = sr.fit(test)

In [ ]:
sr.get_predictions_for_all_users(data, recommender, 10)

In [ ]:
output = _

In [ ]:
output.loc[output['user_song_id'] == output['item_song_id']]

In [ ]:
recs = recommender.recommendForAllUsers(50)

In [ ]:
recs_df = recs.toPandas()

In [ ]:
output = pd.DataFrame(
    columns=['user_song_id', 'item_song_id', 'rating'])

In [ ]:
for row in recs_df.head().itertuples():
    df = pd.DataFrame(row[2], columns=['item_song_id', 'rating'])
    df['user_song_id'] = list([row[1]] * (10 * 5))
    for entry in df.itertuples():
        print(entry[3], entry[1])
        if (entry[3], entry[1]) in existing_pairs:
            df.drop(row[0], axis=0, inplace=True)
    output = pd.concat([output, df[:10]], sort=True)

In [ ]:
output.loc[output['user_song_id'] == output['item_song_id']]

In [ ]:
(22970, 22970) in existing_pairs

In [2]:
cd '/Users/brettashley/galvanize/dsi/00-Capstone/ml-mix-lab/mixmaker/'

/Users/brettashley/galvanize/dsi/00-Capstone/ml-mix-lab/mixmaker


In [3]:
import model

In [4]:
import database_interaction

In [11]:
sr = model.SongRecommender(itemCol='song_id',
                    userCol='sampled_by_song_id',
                    ratingCol='is_connected',
                    nonnegative=True,
                    regParam=0.01,
                    rank=200,
                    alpha=.2)

db = database_interaction.DatabaseInteraction(db_name='mixmaker')

df = db.get_table('connections_distinct')
data = sr.spark.createDataFrame(df)
recommender = sr.fit(data)

In [ ]:
preds = sr.get_predictions_for_all_users(df, recommender, n_predictions=10)

In [ ]:
db.write_predictions(preds, 'predictions')

In [10]:
2129 * 2

4258